In [14]:
# Read the csv dataset
import py_entitymatching as em
A = em.read_csv_metadata('../datasets/yelp_data.csv')
A['ID'] = range(0, len(A))
em.set_key(A, 'ID')

B = em.read_csv_metadata('../datasets/inspection_data.csv')
B['ID'] = range(0, len(B))
em.set_key(B, 'ID')



Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


True

In [15]:
A.head()

,name,address,zipcode,cuisine,ID
0,barn joo 35,"34 w 35th st, ,",10001,korean gastropubs tapas/small plates,0
1,friedman's,"132 w 31st st, ,",10001,american (new) gluten-free breakfast & brunch,1
2,juniper,"237 w 35th st,",10001,cocktail bars american (new),2
3,haymaker bar and kitchen,"252 w 29th st, ,",10001,gastropubs beer bar,3
4,izakaya mew,"53 w 35th st, basement,",10001,sushi bars izakaya cocktail bars,4


## Down-sample

In [16]:
sample_A, sample_B = em.down_sample(A, B, size=18000, y_param=2)
# Save the datasets because it changes every time we perform down-sampling.
em.to_csv_metadata(sample_A, '../datasets/sample_A_generated.csv')
em.to_csv_metadata(sample_B, '../datasets/sample_B_generated.csv')

0%                          100%
[##                            ] | ETA: 00:00:37 | ETA: 00:00:33

KeyboardInterrupt: 

## Blocking

In [ ]:
# Perform blocking
ab = em.AttrEquivalenceBlocker()
C = ab.block_tables(sample_A, sample_B, 'zipcode', 'zipcode', l_output_attrs=['name', 'address', 'zipcode', 'cuisine'], r_output_attrs=['name', 'address', 'zipcode', 'cuisine'])
ob = em.OverlapBlocker()
D = ob.block_candset(C, 'address', 'address', overlap_size=4)

In [ ]:
# Describe the candidate set
print(len(D))
D.head(2)

In [ ]:
# Sample the candidate set.
S = em.sample_table(D, sample_size=1000) # C is the candidate set to be sampled from.
# Write the sample set to a file for further processing.
em.to_csv_metadata(S, '../datasets/candidate_sample_generated.csv')

## Feature extraction

In [205]:
# Loading the saved files
import py_entitymatching as em
sample_A = em.read_csv_metadata('../datasets/sample_A.csv')
sample_B = em.read_csv_metadata('../datasets/sample_B.csv')
S = em.read_csv_metadata('../datasets/candidate_sample.csv')
G = em.read_csv_metadata('../datasets/candidate_sample_labeled.csv')

# Adding metadata to the files
em.set_key(sample_A, 'ID')
em.set_key(sample_B, 'ID')

em.set_key(S, '_id')
em.set_fk_ltable(S, 'ltable_ID')
em.set_fk_rtable(S, 'rtable_ID')
em.set_ltable(S, sample_A)
em.set_rtable(S, sample_B)

em.set_key(G, '_id')
em.set_fk_ltable(G, 'ltable_ID')
em.set_fk_rtable(G, 'rtable_ID')
em.set_ltable(G, sample_A)
em.set_rtable(G, sample_B)
IJ = em.split_train_test(G, train_proportion=0.5, random_state=0)
I = IJ['train']
J = IJ['test']

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


In [403]:
# Automatic Feature Extraction
# match_f = em.get_features_for_matching(sample_A, sample_B)

# Manual Feature Extraction
match_t = em.get_tokenizers_for_matching()
match_s = em.get_sim_funs_for_matching()
atypes1 = em.get_attr_types(A)
atypes2 = em.get_attr_types(B)
match_c = em.get_attr_corres(A, B)
match_f = em.get_features(A, B, atypes1, atypes2, match_c, match_t, match_s)
match_f = match_f.drop([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31])

# Create blackbox features
import re
def custom_address_jaccard(ltuple, rtuple):
    sim = 0.0
    # Extract all numbers from the address
    numbers = re.findall(r'\d+', ltuple['address'])
    num_matches = 0
    for number in numbers:
        if number in rtuple['address']:
            num_matches = num_matches + 1
    if len(numbers) > 0:
        sim = float(num_matches) / float(len(numbers))    
    
    jaccard_sim = em.jaccard(em.tok_alphanumeric(ltuple['address']), em.tok_alphanumeric(rtuple['address']))
    return sim * jaccard_sim * em.exact_match(ltuple['zipcode'], rtuple['zipcode'])

def custom_name_jaccard(ltuple, rtuple):
    jaccard_sim = em.jaccard(em.tok_alphabetic(ltuple['name']), em.tok_alphabetic(rtuple['name']))
    return jaccard_sim

def custom_address_overlap(ltuple, rtuple):    
    sim = em.overlap_coeff(em.tok_alphanumeric(ltuple['address']), em.tok_alphanumeric(rtuple['address']))
    return sim * em.exact_match(ltuple['zipcode'], rtuple['zipcode'])

def custom_name_overlap(ltuple, rtuple):
    overlap_sim = em.overlap_coeff(em.tok_alphabetic(ltuple['name']), em.tok_alphabetic(rtuple['name']))
    return overlap_sim

def custom_address_cosine(ltuple, rtuple):    
    sim = em.cosine(em.tok_alphanumeric(ltuple['address']), em.tok_alphanumeric(rtuple['address']))
    return sim * em.exact_match(ltuple['zipcode'], rtuple['zipcode'])

def custom_name_cosine(ltuple, rtuple):
    sim = em.cosine(em.tok_alphabetic(ltuple['name']), em.tok_alphabetic(rtuple['name']))
    return sim

def custom_cuisine_overlap(ltuple, rtuple):
    sim = em.overlap_coeff(em.tok_alphabetic(ltuple['cuisine']), em.tok_alphabetic(rtuple['cuisine']))
    return max(sim, custom_address_overlap(ltuple, rtuple))

status = em.add_blackbox_feature(match_f, 'custom_address_jaccard', custom_address_jaccard)
status = em.add_blackbox_feature(match_f, 'custom_name_jaccard', custom_name_jaccard)
status = em.add_blackbox_feature(match_f, 'custom_name_overlap', custom_name_overlap)
status = em.add_blackbox_feature(match_f, 'custom_address_overlap', custom_address_overlap)
status = em.add_blackbox_feature(match_f, 'custom_name_cosine', custom_name_cosine)
status = em.add_blackbox_feature(match_f, 'custom_address_cosine', custom_address_cosine)
status = em.add_blackbox_feature(match_f, 'custom_cuisine_overlap', custom_cuisine_overlap)

match_f

,feature_name,left_attribute,right_attribute,left_attr_tokenizer,right_attr_tokenizer,simfunction,function,function_source,is_auto_generated
16,zipcode_zipcode_exm,zipcode,zipcode,None,None,exact_match,<function zipcode_zipcode_exm at 0x17cd2be18>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
1,custom_address_jaccard,None,None,None,None,None,<function custom_address_jaccard at 0x18faf28c8>,None,False
2,custom_name_jaccard,None,None,None,None,None,<function custom_name_jaccard at 0x18faf2bf8>,None,False
3,custom_name_overlap,None,None,None,None,None,<function custom_name_overlap at 0x18faf2c80>,None,False
4,custom_address_overlap,None,None,None,None,None,<function custom_address_overlap at 0x18faf2950>,None,False
5,custom_name_cosine,None,None,None,None,None,<function custom_name_cosine at 0x18faf2268>,None,False
6,custom_address_cosine,None,None,None,None,None,<function custom_address_cosine at 0x18faf2158>,None,False
7,custom_cuisine_overlap,None,None,None,None,None,<function custom_cuisine_overlap at 0x18faf2a60>,None,False


In [404]:
H = em.extract_feature_vecs(I, feature_table=match_f,attrs_after=['label'])
H.head(10)

0%                          100%
[##############################] | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00
Total time elapsed: 00:00:00


,_id,ltable_ID,rtable_ID,zipcode_zipcode_exm,custom_address_jaccard,custom_name_jaccard,custom_name_overlap,custom_address_overlap,custom_name_cosine,custom_address_cosine,custom_cuisine_overlap,label
631,1003068,5272,20785,1,0.000000,0.00,0.000000,0.800000,0.000000,0.800000,0.800000,0
185,469420,2029,20213,1,1.000000,0.00,0.000000,1.000000,0.000000,1.000000,1.000000,0
732,1123677,6975,23836,1,1.000000,0.50,0.666667,1.000000,0.666667,1.000000,1.000000,1
404,830665,3643,12177,1,1.000000,0.25,0.500000,1.000000,0.408248,1.000000,1.000000,1
194,476336,2057,22820,1,1.000000,0.00,0.000000,1.000000,0.000000,1.000000,1.000000,0
564,916386,4525,9904,1,0.000000,0.00,0.000000,0.666667,0.000000,0.666667,0.666667,0
401,826151,3603,11178,1,1.000000,1.00,1.000000,1.000000,1.000000,1.000000,1.000000,1
530,915897,4504,22782,1,0.000000,0.00,0.000000,0.666667,0.000000,0.666667,0.666667,0
656,1031298,5444,11745,1,0.714286,1.00,1.000000,0.833333,1.000000,0.833333,1.000000,1
441,853233,3913,8270,1,0.000000,0.00,0.000000,0.833333,0.000000,0.833333,0.833333,0


## Matching

In [405]:
# Create a set of ML-matchers
dt = em.DTMatcher(name='DecisionTree', random_state=0)
nb = em.NBMatcher(name='NaiveBayes')
svm = em.SVMMatcher(name='SVM', random_state=0)
rf = em.RFMatcher(name='RF', random_state=0)
lg = em.LogRegMatcher(name='LogReg', random_state=0)
ln = em.LinRegMatcher(name='LinReg')

In [406]:
# Report Precision
result = em.select_matcher([dt, nb, rf, svm, ln, lg], table=H, 
        exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'],
        k=5,
        target_attr='label', metric='precision', random_state=0)
result['cv_stats']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DecisionTree,<py_entitymatching.matcher.dtmatcher.DTMatcher object at 0x11b3da7b8>,5,0.754717,0.767442,0.756098,0.837209,0.666667,0.756426
1,NaiveBayes,<py_entitymatching.matcher.nbmatcher.NBMatcher object at 0x11b3da080>,5,0.725490,0.804878,0.767442,0.853659,0.700000,0.770294
2,RF,<py_entitymatching.matcher.rfmatcher.RFMatcher object at 0x11b3f9278>,5,0.750000,0.785714,0.761905,0.857143,0.690476,0.769048
3,SVM,<py_entitymatching.matcher.svmmatcher.SVMMatcher object at 0x11b3da6d8>,5,0.740741,0.804878,0.777778,0.863636,0.690476,0.775502
4,LinReg,<py_entitymatching.matcher.linregmatcher.LinRegMatcher object at 0x11b3f9da0>,5,0.754717,0.804878,0.777778,0.860465,0.700000,0.779568
5,LogReg,<py_entitymatching.matcher.logregmatcher.LogRegMatcher object at 0x11b3f9e80>,5,0.760000,0.837838,0.772727,0.857143,0.700000,0.785542


In [407]:
# Report Recall
# Select the best ML matcher using CV
result = em.select_matcher([dt, nb, rf, svm, ln, lg], table=H, 
        exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'],
        k=5,
        target_attr='label', metric='recall', random_state=0)
result['cv_stats']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DecisionTree,<py_entitymatching.matcher.dtmatcher.DTMatcher object at 0x11b3da7b8>,5,1.000,0.942857,0.837838,0.878049,0.787879,0.889325
1,NaiveBayes,<py_entitymatching.matcher.nbmatcher.NBMatcher object at 0x11b3da080>,5,0.925,0.942857,0.891892,0.853659,0.848485,0.892378
2,RF,<py_entitymatching.matcher.rfmatcher.RFMatcher object at 0x11b3f9278>,5,0.975,0.942857,0.864865,0.878049,0.878788,0.907912
3,SVM,<py_entitymatching.matcher.svmmatcher.SVMMatcher object at 0x11b3da6d8>,5,1.000,0.942857,0.945946,0.926829,0.878788,0.938884
4,LinReg,<py_entitymatching.matcher.linregmatcher.LinRegMatcher object at 0x11b3f9da0>,5,1.000,0.942857,0.945946,0.902439,0.848485,0.927945
5,LogReg,<py_entitymatching.matcher.logregmatcher.LogRegMatcher object at 0x11b3f9e80>,5,0.950,0.885714,0.918919,0.878049,0.848485,0.896233


In [408]:
# Report F1
# Select the best ML matcher using CV
result = em.select_matcher([dt, nb, rf, svm, ln, lg], table=H, 
        exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'],
        k=5,
        target_attr='label', metric='f1', random_state=0)
result['cv_stats']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DecisionTree,<py_entitymatching.matcher.dtmatcher.DTMatcher object at 0x11b3da7b8>,5,0.860215,0.846154,0.794872,0.857143,0.722222,0.816121
1,NaiveBayes,<py_entitymatching.matcher.nbmatcher.NBMatcher object at 0x11b3da080>,5,0.813187,0.868421,0.825000,0.853659,0.767123,0.825478
2,RF,<py_entitymatching.matcher.rfmatcher.RFMatcher object at 0x11b3f9278>,5,0.847826,0.857143,0.810127,0.867470,0.773333,0.831180
3,SVM,<py_entitymatching.matcher.svmmatcher.SVMMatcher object at 0x11b3da6d8>,5,0.851064,0.868421,0.853659,0.894118,0.773333,0.848119
4,LinReg,<py_entitymatching.matcher.linregmatcher.LinRegMatcher object at 0x11b3f9da0>,5,0.860215,0.868421,0.853659,0.880952,0.767123,0.846074
5,LogReg,<py_entitymatching.matcher.logregmatcher.LogRegMatcher object at 0x11b3f9e80>,5,0.844444,0.861111,0.839506,0.867470,0.767123,0.835931


# Debug Matcher

# Find Matcher after debugging

In [409]:
# Train using feature vectors from I
matchers = [dt, nb, rf, svm, ln, lg];
names = ['Decision Tree', 'Naive Bayes', 'Random Forest', 'SVM', 'Linear Regression', 'Logistic Regression']
for i in range(0,len(matchers)):
    matcher = matchers[i]
    name = names[i]
    matcher.fit(table=H, 
           exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'], 
           target_attr='label')

    # Convert J into a set of feature vectors using F
    L = em.extract_feature_vecs(J, feature_table=match_f,attrs_after=['label'])

    # Predict on L 
    predictions = matcher.predict(table=L, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'], 
                  append=True, target_attr='predicted', inplace=False)
    # Evaluate the predictions
    eval_result = em.eval_matches(predictions, 'label', 'predicted')
    print(name)
    em.print_eval_summary(eval_result)

0%                          100%
[##############################] | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00
Total time elapsed: 00:00:00
0%                          100%
[#########                     ] | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00

Decision Tree
Precision : 80.65% (175/217)
Recall : 87.94% (175/199)
F1 : 84.13%
False positives : 42 (out of 217 positive predictions)
False negatives : 24 (out of 283 negative predictions)


[##############################] | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00
Total time elapsed: 00:00:00
0%                          100%
[########                      ] | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00

Naive Bayes
Precision : 80.65% (175/217)
Recall : 87.94% (175/199)
F1 : 84.13%
False positives : 42 (out of 217 positive predictions)
False negatives : 24 (out of 283 negative predictions)


[##############################] | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00
Total time elapsed: 00:00:00
0%                          100%
[#########                     ] | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00

...

SVM
Precision : 81.33% (183/225)
Recall : 91.96% (183/199)
F1 : 86.32%
False positives : 42 (out of 225 positive predictions)
False negatives : 16 (out of 275 negative predictions)


[##############################] | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00
Total time elapsed: 00:00:00
0%                          100%
[########                      ] | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00

Linear Regression
Precision : 80.62% (183/227)
Recall : 91.96% (183/199)
F1 : 85.92%
False positives : 44 (out of 227 positive predictions)
False negatives : 16 (out of 273 negative predictions)


[##############################] | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00
Total time elapsed: 00:00:00


Logistic Regression
Precision : 81.31% (174/214)
Recall : 87.44% (174/199)
F1 : 84.26%
False positives : 40 (out of 214 positive predictions)
False negatives : 25 (out of 286 negative predictions)
